# Drift Example

Simple 1 m drift based on the benchmark in:

3D Space Charge in Bmad
C. E. Mayes, R. D. Ryne,  D. C. Sagan,

https://accelconf.web.cern.ch/ipac2018/papers/thpak085.pdf


In [ ]:
from impact import Impact

In [ ]:
from pmd_beamphysics.units import mec2

import numpy as np
import os


import matplotlib.pyplot as plt

%config InlineBackend.figure_format='retina'

In [ ]:
# locate the drift template
ifile = "../templates/drift/ImpactT.in"
os.path.exists(ifile)

In [ ]:
# calculate gamma*beta
Etot = 10e6  # eV
gamma = Etot / mec2
GB = np.sqrt(gamma**2 - 1)
GB

# Use Impact's built-in Gaussian particle generator

In [ ]:
I = Impact(ifile)
I.header["Np"] = 100000
I.header["Nx"] = 32
I.header["Ny"] = 32
I.header["Nz"] = 32
I.header["Dt"] = 10e-12

In [ ]:
I.lattice

In [ ]:
I.run()

In [ ]:
I.output["stats"].keys()

In [ ]:
PI = I.particles["initial_particles"]
PF = I.particles["final_particles"]

In [ ]:
PF.plot("x", "px")
PF.plot("delta_z", "delta_pz")

# Make particles in distgen

In [ ]:
from distgen import Generator

YAML = """
n_particle: 20000
random_type: hammersley
species: electron
start:
  tstart:
    units: sec
    value: 0
  type: time
total_charge:
  units: nC
  value: 1
r_dist:
  sigma_xy:
    units: mm
    value: 1.0
  type: radial_gaussian
z_dist:
  avg_z:
    units: mm
    value: 0
  sigma_z:
    units: mm
    value: 0.1
  type: gaussian
  

transforms:
  setPz:
    type: set_avg pz
    avg_pz: 
      value: 10
      units: MeV/c
  
"""
G = Generator(YAML)
G.run()
P = G.particles

In [ ]:
I = Impact(ifile, initial_particles=P, verbose=False)
print(I)

In [ ]:
I.header["Nemission"]

In [ ]:
I = Impact(ifile, initial_particles=P)
I.run()
PF2 = I.particles["final_particles"]

In [ ]:
PF2.plot("x", "px")
PF2.plot("delta_z", "delta_pz")

# Compare

In [ ]:
for k in ["x", "px", "z", "pz"]:
    plt.hist(PF[k], density=True, bins=100, label="Impact-T generator", alpha=0.5)
    plt.hist(PF2[k], density=True, bins=100, label="Distgen generator", alpha=0.5)
    plt.xlabel(k)
    plt.legend()
    plt.show()